In [1]:

from tqdm import tqdm
import pandas as pd
import datasets
from pathlib import Path
from transformers import T5ForConditionalGeneration, AutoTokenizer
from typing import List, Optional, Dict
import pickle
import torch
from torch.utils.data import DataLoader

tqdm.pandas()

%load_ext autoreload
%autoreload 2

from pywikidata import Entity
from kbqa.utils.train_eval import get_best_checkpoint_path

from trie import MarisaTrie

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
!nvidia-smi

Failed to initialize NVML: Unknown Error


In [12]:
!git clone https://github.com/askplatypus/wikidata-simplequestions.git

test_df = pd.read_csv(
    "./wikidata-simplequestions/annotated_wd_data_test_answerable.txt",
    sep="\t",
    names=["S", "P", "O", "Q"],
)

train_df = pd.read_csv(
    "./wikidata-simplequestions/annotated_wd_data_train_answerable.txt",
    sep="\t",
    names=["S", "P", "O", "Q"],
)

valid_df = pd.read_csv(
    "./wikidata-simplequestions/annotated_wd_data_valid_answerable.txt",
    sep="\t",
    names=["S", "P", "O", "Q"],
)

fatal: destination path 'wikidata-simplequestions' already exists and is not an empty directory.


In [17]:
train_df.index.size

19481

In [7]:
import numpy as np

filtered_test = np.load("simple_questions_filtered.npy")
test_df = pd.DataFrame(filtered_test, columns=["S", "P", "O", "Q"])
print(test_df.index.size)
test_df.head()

2438


,S,P,O,Q
0,Q154335,P509,Q12152,what was the cause of death of yves klein?
1,Q62498,P21,Q6581097,how does engelbert zaschka identify?
2,Q182485,P413,Q1143358,what position does pee wee reese play in baseb...
3,Q12152,P509,Q6371569,Which Swiss conductor's cause of death is myoc...
4,Q472382,P19,Q23051,what is the place of birth of sam edwards??


In [3]:
tokenizer = AutoTokenizer.from_pretrained("google/t5-large-ssm")

def convert_to_features(example_batch):
    input_encodings = tokenizer(
        example_batch['Q'],
        padding="max_length",
        truncation=True,
        max_length=64,
    )m
    return input_encodings

model = T5ForConditionalGeneration.from_pretrained(
    get_best_checkpoint_path(Path('../../../runs/wdsq_tunned/google_t5-large-ssm/models/'))
)

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_return_sequences=200
num_beams=200
num_beam_groups=20
diversity_penalty=0.1
batch_size=2


In [6]:
!mkdir ./wdsq_dataset/


def prepare_dataset(df, name):
    test_dataset = datasets.Dataset.from_pandas(df)
    test_dataset = test_dataset.map(convert_to_features, batched=True)
    test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

    dataloader = DataLoader(test_dataset, batch_size=batch_size)

    generated_decoded = {f"answer_{idx}": [] for idx in range(num_return_sequences)}
    for batch in tqdm(dataloader, desc="evaluate model"):
        generated_ids = model.generate(
            batch["input_ids"].to(device),
            num_beams=num_beams,
            num_return_sequences=num_return_sequences,
            num_beam_groups=num_beam_groups,
            diversity_penalty=diversity_penalty,
            # prefix_allowed_tokens_fn=prefix_allowed_tokens,
        )
        generated_decoded_batch = tokenizer.batch_decode(
            generated_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False,
        )

        current_batch_size = batch["input_ids"].shape[0]
        for start_batch_idx in range(
            0, current_batch_size * num_return_sequences, num_return_sequences
        ):
            for answer_idx, answer in enumerate(
                generated_decoded_batch[
                    start_batch_idx : start_batch_idx + num_return_sequences
                ]
            ):
                generated_decoded[f"answer_{answer_idx}"].append(answer)

    _df = pd.concat([df, pd.DataFrame(generated_decoded)], axis=1)
    _df.to_csv(f"./wdsq_dataset/{name}_no_prefix_tree.csv", index=False)
    _df.to_pickle(f"./wdsq_dataset/{name}_no_prefix_tree.pkl")


prepare_dataset(train_df, "train")
prepare_dataset(valid_df, "valid")
prepare_dataset(test_df, "filtered_test")

mkdir: cannot create directory ‘./wdsq_dataset/’: File exists


evaluate model:   0%|          | 0/9741 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation_beam_search.py:197: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(
2023-01-09 08:56:30.501473: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in p

KeyboardInterrupt: 

In [6]:
# with open('./wdsq_t5_trie.pkl', 'rb') as f:
#     trie = pickle.load(f)

# def prefix_allowed_tokens(batch_id, sent):
#     result = trie.get(sent.tolist())
#     return result

In [24]:
!git clone https://github.com/s-nlp/wikidata_rubq.git
test_dataset = datasets.load_dataset(
    "wikidata_rubq", "multiple_en", cache_dir=".", ignore_verifications=True
)

tokenizer = AutoTokenizer.from_pretrained("google/t5-large-ssm")


def convert_to_features(example_batch):
    input_encodings = tokenizer(
        example_batch["question"],
        padding="max_length",
        truncation=True,
        max_length=64,
    )
    return input_encodings


test_dataset = test_dataset.map(convert_to_features, batched=True)["test"]
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
fatal: destination path 'wikidata_rubq' already exists and is not an empty directory.


Reusing dataset wikidata_rubq (./wikidata_rubq/multiple_en/0.0.1/8b87b3d5a6c617afe07b22dd53456e86d924379154f567767edfc8653ae0516d)
100%|██████████| 2/2 [00:00<00:00, 32.05ba/s]


In [25]:
model = T5ForConditionalGeneration.from_pretrained(
    get_best_checkpoint_path(
        Path("../../../runs/wdsq_tunned/google_t5-large-ssm/models/")
    )
)

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_return_sequences = 200
num_beams = 200
num_beam_groups = 20
diversity_penalty = 0.1
batch_size = 2

dataloader = DataLoader(test_dataset, batch_size=batch_size)

generated_decoded = {f"answer_{idx}": [] for idx in range(num_return_sequences)}
for batch in tqdm(dataloader, desc="evaluate model"):
    generated_ids = model.generate(
        batch["input_ids"].to(device),
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
        num_beam_groups=num_beam_groups,
        diversity_penalty=diversity_penalty,
        # prefix_allowed_tokens_fn=prefix_allowed_tokens,
    )
    generated_decoded_batch = tokenizer.batch_decode(
        generated_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False,
    )

    current_batch_size = batch["input_ids"].shape[0]
    for start_batch_idx in range(
        0, current_batch_size * num_return_sequences, num_return_sequences
    ):
        for answer_idx, answer in enumerate(
            generated_decoded_batch[
                start_batch_idx : start_batch_idx + num_return_sequences
            ]
        ):
            generated_decoded[f"answer_{answer_idx}"].append(answer)

evaluate model:   0%|          | 0/700 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:1202: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/generation_beam_search.py:197: UserWarning: Passing `max_length` to BeamSearchScorer is deprecated and has no effect. `max_length` should be passed directly to `beam_search(...)`, `beam_sample(...)`, or `group_beam_search(...)`.
  warnings.warn(
2023-01-08 14:38:01.302002: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in pe

In [26]:
# test_with_answers_df = pd.concat([test_df, pd.DataFrame(generated_decoded)], axis=1)
# test_with_answers_df.to_csv('rubq_test_with_answers_no_prefix.csv', index=False)

In [34]:
_df = pd.DataFrame(
    {
        "O": test_dataset["object"],
        "Q": test_dataset["question"],
    }
)

test_with_answers_df = pd.concat([_df, pd.DataFrame(generated_decoded)], axis=1)
test_with_answers_df.to_csv("rubq_test_with_answers_no_prefix.csv", index=False)
test_with_answers_df.head()

,O,Q,answer_0,answer_1,answer_2,answer_3,answer_4,answer_5,answer_6,answer_7,...,answer_190,answer_191,answer_192,answer_193,answer_194,answer_195,answer_196,answer_197,answer_198,answer_199
0,"[Q7944, Q167903, Q5975740, Q60186, Q2580904, Q...",What can cause a tsunami?,tsunami,tsunami,tsunami,tsunami,tsunami,tsunami,tsunami,tsunami,...,actinopteryx,cyclotomis,tsunami,cyclotactic wave,cyclotactic event,cyclosis,tidal wave,taijiquan,Typhoon Haiyan,cyclotomavirus
1,[Q102513],"Who wrote the novel ""uncle Tom's Cabin""?",Harriet Beecher Stowe,Harriet Beecher Stowe,Harriet Beecher Stowe,Harriet Beecher Stowe,Harlan Ellison,Harriet Beecher Stowe,Harlan Ellison,Harriet Beecher Stowe,...,Anne McCaffrey,Harlan Ellison,Harriet Beecher Stowe.,Lynda Benson,Ernest Hemingway,Louisa May Alcott,Lyndall McCord,Haruki Murakami,Harriet Monroe Cooke,Herman Melville
2,[Q692],"Who is the author of the play ""Romeo and Juliet""?",William Shakespeare,William Shakespeare,William Shakespeare,William Shakespeare,William Shakespeare,William Shakespeare,William Shakespeare,William Shakespeare,...,Juliet of the Alps,Jean Racine,Richard Brinsley Shaw,Henry Fielding,Ben Travers,A. R. Toklas,Jean Genet,Juliet Prowse,"Henry Fielding, Jr.",Richard Brinsley Hayes
3,[Q19660],What is the name of the capital of Romania?,Bucharest,Bucharest,Bucharest,Bucharest,Bucharest,Bucharest,Bucharest,Bucharest,...,Ploiești County,Bucharest International Airport,Chişinău,Bucharest City Hall,Chișinău,Cluj-Napoca,Bucharest Capital City,Dobrudja,Bucharest Capital,Ploieşti County
4,"[Q6607, Q17172850, Q483994, Q626035, Q2643890]",What instrument did Jimi Hendrix play?,guitar,guitar,guitar,guitar,guitar,guitar,guitar,guitar,...,elm guitar,Fender Rhodes,MIDI,Fender Telecaster Deluxe,Fender Bass Guitar,banjo,cello,harpy guitar,ukulele,clarinet


In [ ]:
model = T5ForConditionalGeneration.from_pretrained(
    get_best_checkpoint_path(
        Path("../../../runs/wdsq_tunned/google_t5-large-ssm/models/")
    )
)

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_return_sequences = 200
num_beams = 200
num_beam_groups = 20
diversity_penalty = 0.1
batch_size = 2

dataloader = DataLoader(test_dataset, batch_size=batch_size)

generated_decoded = {f"answer_{idx}": [] for idx in range(num_return_sequences)}
for batch in tqdm(dataloader, desc="evaluate model"):
    generated_ids = model.generate(
        batch["input_ids"].to(device),
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
        num_beam_groups=num_beam_groups,
        diversity_penalty=diversity_penalty,
        # prefix_allowed_tokens_fn=prefix_allowed_tokens,
    )
    generated_decoded_batch = tokenizer.batch_decode(
        generated_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False,
    )

    current_batch_size = batch["input_ids"].shape[0]
    for start_batch_idx in range(
        0, current_batch_size * num_return_sequences, num_return_sequences
    ):
        for answer_idx, answer in enumerate(
            generated_decoded_batch[
                start_batch_idx : start_batch_idx + num_return_sequences
            ]
        ):
            generated_decoded[f"answer_{answer_idx}"].append(answer)

test_with_answers_no_prefix_df = pd.concat(
    [test_df, pd.DataFrame(generated_decoded)], axis=1
)
test_with_answers_no_prefix_df.to_csv("test_with_answers_no_prefix.csv", index=False)

In [3]:
import numpy as np

questions = np.load("./questions_test.npy")
answers = np.load("./answers_test.npy", allow_pickle=True)

In [4]:
questions

array(['Что может вызвать цунами?',
       'Кто написал роман «Хижина дяди Тома»?',
       'Кто автор пьесы «Ромео и Джульетта»?', ...,
       'Кто нарисовал черный квадрат?',
       'Кто разработал клеточную теорию?',
       'В каких единицах измеряется масса?'], dtype='<U144')

In [5]:
answers

array([list(['Q7944', 'Q60186', 'Q167903', 'Q2580904', 'Q5975740', 'Q7692360']),
       list(['Q102513']), list(['Q692']), ..., list(['Q130777']),
       list(['Q76432', 'Q76745', 'Q76747']), list(['Q11570'])],
      dtype=object)